## 1. Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

___

## 2. Prepare the Dataset

- Use MNIST as a typical example, normalize it to [-1, 1] for better GAN performance

This command creates a **data transformation pipeline** typically used in image preprocessing for deep learning tasks with libraries like PyTorch.

Here’s what the command does, step-by-step:

1. **`transforms.Compose([...])`**  
   This wraps multiple transformations into one. It applies them sequentially to the input data.

2. **`transforms.ToTensor()`**  
   Converts a PIL image or NumPy ndarray into a PyTorch tensor. It also scales pixel values from the range  to [0.0, 1.0].

3. **`transforms.Normalize(mean=(0.5,), std=(0.5,))`**  
   Normalizes the tensor by subtracting the mean and dividing by the standard deviation for each channel. Here, the single value `(0.5,)` corresponds to one channel (like grayscale).

   Since the pixel values are between 0 and 1 after `ToTensor()`, normalization with mean=0.5 and std=0.5 converts them roughly to the range [-1, 1] using this formula:

   $$
   \text{normalized\_pixel} = \frac{\text{pixel} - 0.5}{0.5}
   $$

### Purpose of this command:

- **Convert images to tensors** (making them compatible with PyTorch models).
- **Normalize pixel intensity values** so the model trains more effectively (helps with convergence and stabilizes training by standardizing input).

This transformation is common in training neural networks on datasets like MNIST or grayscale images since it processes the image into a normalized tensor input suitable for the model.

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

This command creates a **training dataset object** for the MNIST dataset using PyTorch’s `torchvision.datasets` module.

Here’s a breakdown of the command:

- **`datasets.MNIST(...)`**  
  This loads the MNIST handwritten digit dataset, which contains grayscale images of digits (0–9).

- **`root='./data'`**  
  Specifies the directory where the MNIST data will be stored or loaded from. If the data is not already present, it will be downloaded to this folder.

- **`train=True`**  
  Indicates that this dataset object should load the **training split** of MNIST (60,000 images). If set to `False`, it would load the test split (10,000 images).

- **`download=True`**  
  If the dataset isn’t already present in the specified `root` folder, PyTorch will automatically download it from the internet.

- **`transform=transform`**  
  Applies the previously defined transformations (such as conversion to tensor and normalization) to every image when it is loaded. This preprocesses the images so they're ready for training.

### Purpose of this command:

- It **loads** the MNIST training dataset, downloading it if necessary.
- It **applies the defined transformations** (like converting images to tensors and normalizing) on the data.
- It prepares the dataset in a format suitable for training a PyTorch model.

In short, this command **sets up your training data pipeline** for feeding images into a neural network.

In [3]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

This command creates a **DataLoader** object from the previously defined `train_dataset` (MNIST training dataset in this case).

Here’s what it does:

- **`DataLoader(train_dataset, batch_size=64, shuffle=True)`**  
  This wraps the dataset in a convenient iterator for training machine learning models.

- **`batch_size=64`**  
  Specifies that data will be loaded in batches of 64 samples. Instead of passing one image at a time, the model receives 64 images in each training step, which is efficient and stabilizes training.

- **`shuffle=True`**  
  Means the dataset will be shuffled at the start of each epoch. Shuffling helps the model see the data in a different order each time, which improves training by reducing overfitting and increasing generalization.

### Purpose of this command:

- To **batch the dataset** enabling efficient processing and training.
- To **shuffle the training data** so the model does not learn the order of data, which promotes better generalization.
- To provide an **easily iterable object** that returns batches of transformed images and their labels, ready for training.

In summary, this command sets up the training pipeline to feed the model mini-batches of shuffled data for training. It abstracts away loading and batching complexities so the training loop can run smoothly and efficiently.

In [4]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

___

## 3. Define the Generator

- **Generator:** Turns noisy random input into a 28x28 image.

In [5]:
class Generator(nn.Module):
    def __init__(self, noise_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 784),
            nn.Tanh()  # Output between -1 and 1
        )
    def forward(self, x):
        return self.model(x).view(-1, 1, 28, 28)

___